***GENERATED CODE FOR 2aprsuperstores PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import pyspark
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Order_Month', 'transformation_label': 'String Indexer'}], 'feature': 'Order_Month', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'April', 'max': 'September', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Order_Month'}, {'feature_label': 'Order_Month', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Order_Month')

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        return df


***AUTOML FUNCTIONS***

In [ ]:
from prophet import Prophet
import pandas as pd


def driverProphet(df):
    df.columns = ['ds', 'y']
    df['ds'] = df['ds'].astype(str).str[:-6]
    df['ds'] = pd.to_datetime(df['ds'])
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=365)
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    m.plot(forecast)
    return m


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run 2aprsuperstoresHooks.ipynb
try:
	#sourcePreExecutionHook()

	aprsuperstoreshdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1712056667.csv', 'filename': 'tempFile1712056667.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(aprsuperstoreshdfs)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run 2aprsuperstoresHooks.ipynb
try:
	#transformationPreExecutionHook()

	aprsuperstoresfeatureforecast = TransformationTimeSeriesForecastingMain.run(aprsuperstoreshdfs, json.dumps( {"FE": {"dataPercentage": 10, "featureList": [{"transformationsData": [{}], "feature": "Order_Date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Order_Date"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1332.27", "stddev": "2002.49", "min": "3.816", "max": "28106.716", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sales"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_First Class", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_First Class"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_Same Day", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_Same Day"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_Second Class", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.31", "stddev": "1.92", "min": "0.0", "max": "11.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_Second Class"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_Standard Class", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "4.32", "stddev": "3.7", "min": "0.0", "max": "19.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_Standard Class"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Segment_Consumer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.95", "stddev": "3.05", "min": "0.0", "max": "16.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Segment_Consumer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Segment_Corporate", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.62", "stddev": "2.16", "min": "0.0", "max": "10.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Segment_Corporate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Segment_Home Office", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.06", "stddev": "1.74", "min": "0.0", "max": "13.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Segment_Home Office"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_Central", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.33", "stddev": "1.92", "min": "0.0", "max": "10.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_Central"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_East", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.64", "stddev": "2.2", "min": "0.0", "max": "14.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_East"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_South", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.66", "stddev": "1.41", "min": "0.0", "max": "9.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_South"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_West", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.99", "stddev": "2.36", "min": "0.0", "max": "12.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_West"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Category_Furniture", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.2", "stddev": "1.31", "min": "0.0", "max": "6.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Category_Furniture"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Category_Office Supplies", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.38", "stddev": "2.81", "min": "0.0", "max": "14.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Category_Office Supplies"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Category_Technology", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.04", "stddev": "1.18", "min": "0.0", "max": "7.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Category_Technology"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Order_Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2016.37", "stddev": "1.13", "min": "2015", "max": "2018", "missing": "0"}, "updatedLabel": "Order_Year"}, {"transformationsData": [{"feature_label": "Order_Month", "transformation_label": "String Indexer"}], "feature": "Order_Month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "April", "max": "September", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Order_Month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Order_MonthNumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "6.23", "stddev": "3.42", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "Order_MonthNumber"}, {"feature": "Order_Month_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "5.11", "stddev": "3.41", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "Order_Month_stringindexer_transform"}], "features": {"timecolumn": "Order_Date", "tocompare": "Sales"}, "functionList": [{"function": "rollingmean"}], "originalfile": "/FileStore/platform/extra/tempFile1712056667.csv", "statFunction": {"function": "Original", "parameter": ""}}}))

	#transformationPostExecutionHook(aprsuperstoresfeatureforecast)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run 2aprsuperstoresHooks.ipynb
try:
	#mlPreExecutionHook()

	model = driverProphet(aprsuperstoresfeatureforecast)

	#mlPostExecutionHook(model)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
try:
        future = model.make_future_dataframe(periods=30,freq='MS',include_history=False)
        predictedDataFrame = model.predict(future)
        display(model.plot(predictedDataFrame))
        display(model.plot_components(predictedDataFrame))
        display(predictedDataFrame)
except Exception as ex:
    logging.error(ex)

spark.stop()


